# Order Latency Data

To obtain more realistic backtesting results, accounting for latencies is crucial. Therefore, it's important to collect both feed data and order data with timestamps to measure your order latency. The best approach is to gather your own order latencies. You can collect order latency based on your live trading or by regularly submitting orders at a price that cannot be filled and then canceling them for recording purposes. However, if you don't have access to them or want to establish a target, you will need to artificially generate order latency. You can model this latency based on factors such as feed latency, trade volume, and the number of events. In this guide, we will demonstrate a simple method to generate order latency from feed latency using a multiplier and offset for adjustment.

First, loads the feed data.

In [3]:
import numpy as np

data = np.load(r'D:\gptcv\hftbacktest\Data\1000bonkusdt\1000bonkusdt_20240730.npz')['data']
data

array([(3758096385, 1722346072115000000, 1722346072148592385, 0.026995, 51594., 0, 0, 0.),
       (3489660929, 1722346072115000000, 1722346072148592385, 0.027014, 12031., 0, 0, 0.),
       (3489660929, 1722346072115000000, 1722346072148592385, 0.027016,  6919., 0, 0, 0.),
       ...,
       (3758096385, 1722383999959000000, 1722383999965999659, 0.025927, 94562., 0, 0, 0.),
       (3489660929, 1722383999959000000, 1722383999965999659, 0.025951, 69196., 0, 0, 0.),
       (3489660929, 1722383999959000000, 1722383999965999659, 0.025953, 22236., 0, 0, 0.)],
      dtype=[('ev', '<u8'), ('exch_ts', '<i8'), ('local_ts', '<i8'), ('px', '<f8'), ('qty', '<f8'), ('order_id', '<u8'), ('ival', '<i8'), ('fval', '<f8')])

For easy manipulation, converts it into a DataFrame.

In [4]:
import polars as pl

df = pl.DataFrame(data)
df

ev,exch_ts,local_ts,px,qty,order_id,ival,fval
u64,i64,i64,f64,f64,u64,i64,f64
3758096385,1722346072115000000,1722346072148592385,0.026995,51594.0,0,0,0.0
3489660929,1722346072115000000,1722346072148592385,0.027014,12031.0,0,0,0.0
3489660929,1722346072115000000,1722346072148592385,0.027016,6919.0,0,0,0.0
3489660929,1722346072115000000,1722346072148592385,0.027021,142726.0,0,0,0.0
3489660929,1722346072115000000,1722346072148592385,0.027059,81137.0,0,0,0.0
…,…,…,…,…,…,…,…
3758096385,1722383999959000000,1722383999965999659,0.02464,2029.0,0,0,0.0
3758096385,1722383999959000000,1722383999965999659,0.025314,206.0,0,0,0.0
3758096385,1722383999959000000,1722383999965999659,0.025927,94562.0,0,0,0.0


Selects only the events that have both a valid exchange timestamp and a valid local timestamp to get feed latency.

In [5]:
from hftbacktest import EXCH_EVENT, LOCAL_EVENT

df = df.filter((pl.col('ev') & EXCH_EVENT == EXCH_EVENT) & (pl.col('ev') & LOCAL_EVENT == LOCAL_EVENT))

Reduces the number of rows by resampling to approximately 1-second intervals.

In [6]:
df = df.with_columns(
    pl.col('local_ts').alias('ts')
).group_by_dynamic(
    'ts', every='1000000000i'
).agg(
    pl.col('exch_ts').last(),
    pl.col('local_ts').last()
).drop('ts')

df

exch_ts,local_ts
i64,i64
1722346072277000000,1722346072355279990
1722346073936000000,1722346073962855823
1722346074952000000,1722346074967117346
1722346075953000000,1722346075966054542
1722346076944000000,1722346076948889529
…,…
1722383995936000000,1722383995950003153
1722383996995000000,1722383996998491937
1722383997952000000,1722383997993859276


Converts back to the structured NumPy array.

In [7]:
data = df.to_numpy(structured=True)
data

array([(1722346072277000000, 1722346072355279990),
       (1722346073936000000, 1722346073962855823),
       (1722346074952000000, 1722346074967117346), ...,
       (1722383997952000000, 1722383997993859276),
       (1722383998927000000, 1722383998965713810),
       (1722383999959000000, 1722383999965999659)],
      dtype=[('exch_ts', '<i8'), ('local_ts', '<i8')])

Generates order latency. Order latency consists of two components: the latency until the order request reaches the exchange's matching engine and the latency until the response arrives backto the localy. Order latency is not the same as feed latency and does not need to be proportional to feed latency. However, for simplicity, we model order latency to be proportional to feed latency using a multiplier and offset.

In [8]:
mul_entry = 4
offset_entry = 0

mul_resp = 3
offset_resp = 0

order_latency = np.zeros(len(data), dtype=[('req_ts', 'i8'), ('exch_ts', 'i8'), ('resp_ts', 'i8'), ('_padding', 'i8')])
for i, (exch_ts, local_ts) in enumerate(data):
    feed_latency = local_ts - exch_ts
    order_entry_latency = mul_entry * feed_latency + offset_entry
    order_resp_latency = mul_resp * feed_latency + offset_resp

    req_ts = local_ts
    order_exch_ts = req_ts + order_entry_latency
    resp_ts = order_exch_ts + order_resp_latency
    
    order_latency[i] = (req_ts, order_exch_ts, resp_ts, 0)
    
order_latency

array([(1722346072355279990, 1722346072668399950, 1722346072903239920, 0),
       (1722346073962855823, 1722346074070279115, 1722346074150846584, 0),
       (1722346074967117346, 1722346075027586730, 1722346075072938768, 0),
       ...,
       (1722383997993859276, 1722383998161296380, 1722383998286874208, 0),
       (1722383998965713810, 1722383999120569050, 1722383999236710480, 0),
       (1722383999965999659, 1722383999993998295, 1722384000014997272, 0)],
      dtype=[('req_ts', '<i8'), ('exch_ts', '<i8'), ('resp_ts', '<i8'), ('_padding', '<i8')])

In [9]:
df_order_latency = pl.DataFrame(order_latency)
df_order_latency

req_ts,exch_ts,resp_ts,_padding
i64,i64,i64,i64
1722346072355279990,1722346072668399950,1722346072903239920,0
1722346073962855823,1722346074070279115,1722346074150846584,0
1722346074967117346,1722346075027586730,1722346075072938768,0
1722346075966054542,1722346076018272710,1722346076057436336,0
1722346076948889529,1722346076968447645,1722346076983116232,0
…,…,…,…
1722383995950003153,1722383996006015765,1722383996048025224,0
1722383996998491937,1722383997012459685,1722383997022935496,0
1722383997993859276,1722383998161296380,1722383998286874208,0


Checks if latency has invalid negative values.

In [10]:
order_entry_latency = df_order_latency['exch_ts'] - df_order_latency['req_ts']
order_resp_latency = df_order_latency['resp_ts'] - df_order_latency['exch_ts']

In [11]:
(order_entry_latency <= 0).sum()

0

In [12]:
(order_resp_latency <= 0).sum()

0

Here, we wrap the entire process into a method with `njit` for increased speed.

In [21]:
import numpy as np
from numba import njit
import polars as pl
from hftbacktest import LOCAL_EVENT, EXCH_EVENT

@njit
def generate_order_latency_nb(data, order_latency, mul_entry, offset_entry, mul_resp, offset_resp):
    for i in range(len(data)):
        exch_ts = data[i].exch_ts
        local_ts = data[i].local_ts
        feed_latency = local_ts - exch_ts
        order_entry_latency = mul_entry * feed_latency + offset_entry
        order_resp_latency = mul_resp * feed_latency + offset_resp

        req_ts = local_ts
        order_exch_ts = req_ts + order_entry_latency
        resp_ts = order_exch_ts + order_resp_latency

        order_latency[i].req_ts = req_ts
        order_latency[i].exch_ts = order_exch_ts
        order_latency[i].resp_ts = resp_ts

def generate_order_latency(feed_file, output_file = None, mul_entry = 1, offset_entry = 0, mul_resp = 1, offset_resp = 0):
    data = np.load(feed_file)['data']
    df = pl.DataFrame(data)
    
    df = df.filter(
        (pl.col('ev') & EXCH_EVENT == EXCH_EVENT) & (pl.col('ev') & LOCAL_EVENT == LOCAL_EVENT)
    ).with_columns(
        pl.col('local_ts').alias('ts')
    ).group_by_dynamic(
        'ts', every='1000000000i'
    ).agg(
        pl.col('exch_ts').last(),
        pl.col('local_ts').last()
    ).drop('ts')
    
    data = df.to_numpy(structured=True)

    order_latency = np.zeros(len(data), dtype=[('req_ts', 'i8'), ('exch_ts', 'i8'), ('resp_ts', 'i8'), ('_padding', 'i8')])
    
    output_file = r'D:\gptcv\hftbacktest\latency\feed_latency_20240730.npz'
    generate_order_latency_nb(data, order_latency, mul_entry, offset_entry, mul_resp, offset_resp)
    print(order_latency)
    if output_file is not None:
        print(order_latency)
        np.savez_compressed(output_file, data=order_latency)

    return order_latency

In [22]:
order_latency = generate_order_latency(r'D:\gptcv\hftbacktest\Data\1000bonkusdt\1000bonkusdt_20240730.npz', output_file=r'D:\gptcv\hftbacktest\latency\feed_latency_20240730.npz', mul_entry=4, mul_resp=3)

[(1722346072355279990, 1722346072668399950, 1722346072903239920, 0)
 (1722346073962855823, 1722346074070279115, 1722346074150846584, 0)
 (1722346074967117346, 1722346075027586730, 1722346075072938768, 0) ...
 (1722383997993859276, 1722383998161296380, 1722383998286874208, 0)
 (1722383998965713810, 1722383999120569050, 1722383999236710480, 0)
 (1722383999965999659, 1722383999993998295, 1722384000014997272, 0)]
[(1722346072355279990, 1722346072668399950, 1722346072903239920, 0)
 (1722346073962855823, 1722346074070279115, 1722346074150846584, 0)
 (1722346074967117346, 1722346075027586730, 1722346075072938768, 0) ...
 (1722383997993859276, 1722383998161296380, 1722383998286874208, 0)
 (1722383998965713810, 1722383999120569050, 1722383999236710480, 0)
 (1722383999965999659, 1722383999993998295, 1722384000014997272, 0)]
